# Цель задания

Собрать новый датасет с помощью парсинга данных.

## Формат сдачи

Пришлите ссылку на репозиторий, в котором находятся:
* Jupiter Notebook с кодом.
* Итоговый датасет — файл в формате .csv.

**Критерии оценки**:

* Датасет содержит все необходимые поля; размер датасета соответствует эталонному на 90% и более, обучена модель, отправлен сабмишн и выводы — 5 баллов.
* Датасет содержит все необходимые поля; размер датасета соответствует эталонному менее чем на 90%, правильная логика на этапах матчинга, парсинга доп. характеристик авто и мерджа с исходным датасетом — 4 балла.
* Датасет содержит все или почти все необходимые поля; размер датасета соответствует эталонному менее чем на 90% допущена серьезная ошибка на одном из этапов: матчинг, парсинг доп. характеристик авто и мердж с исходным датасетом — 3 балла.
* Получилось спарсить ссылки на модели автомобилей — 2 балла.
* Код не исполняется; нет датасета — 1 балл.

# <center> 🤼‍♀️ Разминаемся (Задание 1)
В качестве первого задания вам предстоит достать значения средних зарплат по городам России. Сайт, на котором они хранятся: https://stepik.org/media/attachments/lesson/866758/mean_salary_by_city.html

Ответ - датафрейм, котором города идут в алфавитном порядке.

Пример ответа:

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image1.png?raw=true' width="550" >

In [97]:
from lxml import etree, html as lhtml
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [2]:
url = 'https://stepik.org/media/attachments/lesson/866758/mean_salary_by_city.html'
response = requests.get(url)
content = response.content

tree = lhtml.fromstring(content)

In [3]:
salary_info = {
    'city': tree.xpath('//div[starts-with(@class, "reg_name")]/a/text()'),
    'mean_salary': tree.xpath('//div/span[starts-with(@class, "reg_salary")]/text()'),
}

df = pd.DataFrame(salary_info, index=None)

def clean_string(s):
    s =  ''.join(s.split()) if isinstance(s, str) else s
    s = s[:-1]
    return s

# Применение функции ко всем строкам всех столбцов
df['mean_salary'] = df['mean_salary'].map(clean_string)
df['mean_salary'] = df['mean_salary'].astype('int')
df.head()

,city,mean_salary
0,Анадырь,129200
1,Москва,113600
2,Салехард,106400
3,Южно-Сахалинск,99000
4,Магадан,95200


# <center> 🌍 Парсим [automobili.ru](https://automobili.ru/cars/catalog/) 🚗

В этой задаче вам предстоит спарсить дополнительные данные по моделям машин с сайта: https://automobili.ru/cars/catalog/ и добавить их в существующий датасет. Для того чтобы было проще понимать, в каком месте вы ошибаетесь (если такое происходит), мы разбили эту задачу на несколько степов, где постепенно будем парсить данный сайт.

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image2.png?raw=true' width="750" >

## <center> 🕊 Собираем ссылки 🔗

Для того чтобы начать доставать информацию о моделях, нужно вначале получить ссылки на эти модели. На первой странице сайта находится список всех марок. Вам нужно:

1) Собрать все ссылки на автомобильные бренды (средствами автоматического парсинга).

2) Пройтись по каждой из полученных ссылок и спарсить названия конкретных моделей автомобилей.

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image3.png?raw=true' width="750" >
<left> <img src='https://github.com/PeMikj/images/blob/main/images/image4.png?raw=true' width="750" >

В итоге у вас должно найтись 325 ссылок, которые выглядят примерно так:

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image5.png?raw=true' width="750" >

Ответ - датафрейм с ссылками и названием модели. Датафрейм должен быть отсортирован по названию модели, а затем по ссылкам.
`df.sort_values(by=['model', 'link'])`

Пример:

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image6.png?raw=true' width="750" >

In [104]:
# URL страницы с автомобильными брендами
url = 'https://automobili.ru/cars/catalog/'

# Отправляем запрос к странице
response = requests.get(url)

# Проверяем, что запрос успешен
if response.status_code == 200:
    # Создаем объект BeautifulSoup для парсинга HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Находим все ссылки на автомобильные бренды
    links = soup.find_all('a', href=True)  # Поиск всех ссылок
    
    # Фильтруем ссылки, чтобы оставить только те, которые ведут на автомобильные бренды
    brand_links = [link['href'] for link in links if '/cars/catalog' in link['href'] and '?letter' not in link['href']]
    # brand_links = [urljoin(url, link['href']) for link in links if '/cars/catalog' in link['href'] and '?letter' not in link['href']]

    # Выводим найденные ссылки
    for link in brand_links:
        print(link)
else:
    print(f'Ошибка при запросе страницы: {response.status_code}')

/cars/catalog/
/cars/catalog/aston_martin/
/cars/catalog/audi/
/cars/catalog/bentley/
/cars/catalog/bmw/
/cars/catalog/bugatti/
/cars/catalog/cadillac/
/cars/catalog/changan/
/cars/catalog/chery/
/cars/catalog/cheryexeed/
/cars/catalog/chevrolet/
/cars/catalog/citroen/
/cars/catalog/dongfeng/
/cars/catalog/faw/
/cars/catalog/ferrari/
/cars/catalog/fiat/
/cars/catalog/geely/
/cars/catalog/genesis/
/cars/catalog/haval/
/cars/catalog/honda/
/cars/catalog/hyundai/
/cars/catalog/infiniti/
/cars/catalog/isuzu/
/cars/catalog/jac/
/cars/catalog/jaguar/
/cars/catalog/jeep/
/cars/catalog/kia/
/cars/catalog/lada/
/cars/catalog/lamborghini/
/cars/catalog/land_rover/
/cars/catalog/lexus/
/cars/catalog/lifan/
/cars/catalog/lotus/
/cars/catalog/maserati/
/cars/catalog/mazda/
/cars/catalog/mercedes-benz/
/cars/catalog/mini/
/cars/catalog/mitsubishi/
/cars/catalog/nissan/
/cars/catalog/opel/
/cars/catalog/peugeot/
/cars/catalog/porsche/
/cars/catalog/renault/
/cars/catalog/rolls-royce/
/cars/catalog/sk

## <center> 👉👈 Матчим ссылки с датасетом

Отлично! Мы получили ссылки на все модели машин, которые есть на сайте, но нам понадобится только небольшая часть, так как многие модели отсутствуют в изначальном датасете. В этом задании вам предстоит сопоставить ссылки и машины из датасета `quickstart_train.csv`.

In [ ]:
import pandas as pd

path = 'https://stepik.org/media/attachments/lesson/866758/quickstart_train.csv'

df = pd.read_csv(path)
df.head(3)

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,109.99,another_bug,4.737759,1.214131e+07,0.1,180.855726,0.023174,174,170
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,34.48,electro_bug,4.480517,1.803909e+07,0.0,187.862734,12.306011,174,174
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,34.93,gear_stick,4.768391,1.588366e+07,0.1,102.382857,2.513319,174,173


<left> <img src='https://github.com/PeMikj/images/blob/main/images/image7.png?raw=true' width="750" >
<left> <img src='https://github.com/PeMikj/images/blob/main/images/image8.png?raw=true' width="300" >



Далее нам нужно сматчить названия моделей машин в исходном и спаршенном датафреймах.
Для решения задачи можно использовать разные способы измерения близости между двумя строками.

Мы будем использовать алгоритм  нахождения наибольшей общей подпоследовательности - [википедия.](https://ru.wikipedia.org/wiki/%D0%9D%D0%B0%D0%B8%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%B0%D1%8F_%D0%BE%D0%B1%D1%89%D0%B0%D1%8F_%D0%BF%D0%BE%D0%B4%D0%BF%D0%BE%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D1%8C)

Для каждой модели в нашем исходном датафрейме нужно:

1) Найти строку с названием модели в напаршенных данных с максимальным значением наибольшей общей подпоследовательности.

2) Нормализовать значение наибольшей общей подпоследовательности на длину строки в исходном датафрейме.

3) Отсечь те случаи, где нормализованное значение меньше 0.85.

4) Если не удалось найти матч - заполняем np.nan.


P.S. Также нужно заменить `vw` на `volkswagen` в изначальном датасете, чтоб было больше совпадений. Нужно понимать, что идеально сматчить не всегда получится, поэтому иногда приходится прибегать к эвристикам, но мы в данном задании этого делать не будем.

Пример матчинга между названиями машин в исходном датафрейме и напаршенном.
Для каждого названия авто из исходного датафрейма находится название из напаршенного с наибольшим значением наибольшей общей подпоследовательности.

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image9.png?raw=true' width="550" >

## <center> ⚗️ Достаем технические характеристики

Достаем информацию о машинах

Отлично! Ссылки мы достали, теперь пришло время получить необходимые данные из них. Это скриншот того, как выглядит страничка сайта для конкретной модели (в нашем случае Renault Sandero):

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image10.png?raw=true' width="750" >


Отсюда вам необходимо для каждой модели получить следующую информацию:

1) `year` - год начала выпуска модели (целое число);
2) `mod` - название модификации;
3) `price` - рекомендованная цена (целое число);
4) `engine` - тип двигателя;
5) `power` - мощность в л.с.;
6) `box` - тип коробки передач;
7) `trans` - тип трансмиссии;
8) `body` - тип кузова;


Это все можно сделать при помощи BeautifulSoap.

Мы берем значения только для самой первой модификации!

Далее соединяем с нашим исходным датасетом. `model` - столбик по которому мы соединяем датасеты.

Получившийся датасет - решение задачи (не изменяйте порядок строк в изначальном датасете).

Пример результата (для удобства представления в исходном датасете сохранены только колонки `car_id` и `model`, но вам нужны все колонки из исходного датасета):

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image11.png?raw=true' width="850" >

## <center> 🏋️‍♂️Тренировка с новыми данными

Обучите модель на обогащенном датасете и сравните результат с предыдущими.

Отправьте сабмишн на kaggle.

Сделайте выводы.